In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fraudulent-transactions-data/Fraud.csv


In [3]:
df = pd.read_csv("/kaggle/input/fraudulent-transactions-data/Fraud.csv")

In [4]:
df.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [5]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


# EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
isFraud = df[df["isFraud"]==1]

## Checking step
nothing of use

In [ ]:
top_step = isFraud["step"].value_counts().iloc[:10]
# top10_step = top10_step.reset_index() # to use matplotlib
top_step
top_step.plot(kind = "bar")

## Checking type
Only these type have fruad transactions
* Cash_out
* Transfer

In [ ]:
df.groupby(['type', 'isFraud']).size().unstack(fill_value=0)

## Checking amount size
Major fraud is observed with 10M value of transaction. After which its random

In [ ]:
top_amount = isFraud[["amount", "type"]].value_counts().iloc[:10]
top_amount

In [ ]:
plt.hist(isFraud[["amount"]])

In [ ]:
plt.hist(isFraud[["amount"]][isFraud["amount"]<=0.2e7])

## Checking nameDest
Making new column to distinguish between merchant and normal recipient.\
There is no fraud if recipient is Merchant.

In [11]:
df["init_nameDest"] = df["nameDest"].apply(lambda x: str(x)[0])
isFraud = df[df["isFraud"]==1]

In [ ]:
df[['init_nameDest', "isFraud"]].value_counts()

In [ ]:
df.groupby(['init_nameDest', "isFraud"]).size().unstack(fill_value=0)

## Balance Sheet

In [ ]:
isFraud[["oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", "type"]].value_counts().iloc[10:1000].reset_index().describe()

In [ ]:
df[(df["isFlaggedFraud"]==1) & (df["isFraud"]==1)]

## Benford’s Law
<!-- ![image.png](attachment:6d4a24ff-6229-491e-8173-a509729ab935.png) -->
<img src="attachment:6d4a24ff-6229-491e-8173-a509729ab935.png" alt="image" width ="600">

In [ ]:
round(df["amount"].apply(lambda x: int(str(x)[0])).value_counts()/df.shape[0]*100,2)

In [ ]:
round(isFraud["amount"].apply(lambda x: int(str(x)[0])).value_counts()/isFraud.shape[0]*100,2)

### Creating new column for first digit of the amount transaction

In [12]:
df["first_amount_digit"] = df["amount"].apply(lambda x: int(str(x)[0]))

In [ ]:
df

# ADASYN Minority Oversampling
### *Warning: For ADASYN Oversampling, required numpy version is >2.3, which is not compatible with matplotlib. After EDA (matplotlib use) run the following commands and restart the session without running EDA* 

In [7]:
!pip install -U numpy
!pip install -U scikit-learn
!pip install -U imbalanced-learn
import sklearn, imblearn
from sklearn.metrics import r2_score

# Checking packages' version
print("NumPy:", np.__version__)
print("scikit-learn:", sklearn.__version__)
print("imblearn:", imblearn.__version__)

  Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.7 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.1 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  A

In [19]:
# Run these lines to get additionally created columns
df["init_nameDest"] = df["nameDest"].apply(lambda x: str(x)[0])
df["first_amount_digit"] = df["amount"].apply(lambda x: int(str(x)[0]))

## Getting X and y

In [13]:
X = df.drop(["isFraud", "isFlaggedFraud", "nameOrig", "nameDest"], axis=1)
X

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,init_nameDest,first_amount_digit
0,1,PAYMENT,9839.64,170136.00,160296.36,0.00,0.00,M,9
1,1,PAYMENT,1864.28,21249.00,19384.72,0.00,0.00,M,1
2,1,TRANSFER,181.00,181.00,0.00,0.00,0.00,C,1
3,1,CASH_OUT,181.00,181.00,0.00,21182.00,0.00,C,1
4,1,PAYMENT,11668.14,41554.00,29885.86,0.00,0.00,M,1
...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,339682.13,0.00,0.00,339682.13,C,3
6362616,743,TRANSFER,6311409.28,6311409.28,0.00,0.00,0.00,C,6
6362617,743,CASH_OUT,6311409.28,6311409.28,0.00,68488.84,6379898.11,C,6
6362618,743,TRANSFER,850002.52,850002.52,0.00,0.00,0.00,C,8


In [14]:
X = pd.get_dummies(X, columns = ["type", "init_nameDest"], drop_first = True)
X

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,first_amount_digit,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,init_nameDest_M
0,1,9839.64,170136.00,160296.36,0.00,0.00,9,False,False,True,False,True
1,1,1864.28,21249.00,19384.72,0.00,0.00,1,False,False,True,False,True
2,1,181.00,181.00,0.00,0.00,0.00,1,False,False,False,True,False
3,1,181.00,181.00,0.00,21182.00,0.00,1,True,False,False,False,False
4,1,11668.14,41554.00,29885.86,0.00,0.00,1,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,0.00,339682.13,3,True,False,False,False,False
6362616,743,6311409.28,6311409.28,0.00,0.00,0.00,6,False,False,False,True,False
6362617,743,6311409.28,6311409.28,0.00,68488.84,6379898.11,6,True,False,False,False,False
6362618,743,850002.52,850002.52,0.00,0.00,0.00,8,False,False,False,True,False


In [15]:
X.columns

Index(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'first_amount_digit', 'type_CASH_OUT', 'type_DEBIT',
       'type_PAYMENT', 'type_TRANSFER', 'init_nameDest_M'],
      dtype='object')

In [16]:
y = df['isFraud']
y

0          0
1          0
2          1
3          1
4          0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 6362620, dtype: int64

In [17]:
# imblearn require numpy > 2.3, this version of numpy will not work with matplot lib, which need 2.1 or 2.2. Keep that in mind
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN

In [20]:
print("Before ADASYN:", Counter(y))

Before ADASYN: Counter({0: 6354407, 1: 8213})


In [21]:
ada = ADASYN(sampling_strategy='auto', random_state=42, n_neighbors=5)
X_res, y_res = ada.fit_resample(X, y)   #Both are pandas dataframe and series respectively

In [22]:
print("After ADASYN:", Counter(y_res)) # Almost balanced

After ADASYN: Counter({0: 6354407, 1: 6353883})


# XGBoost

## Splitting Validation and Test data

In [23]:
# As the dataset is huge (not for XGBoost for the GPLearn), we will pick up 20% of the whole data from which we will plit the train and test X & y.

# Getting 20% of data for faster testing. Rest 80% we can use in validating
from sklearn.model_selection import train_test_split
X_trim, X_val, y_trim, y_val = train_test_split(
    X_res, y_res, test_size=0.9, random_state=21
)

# Train-test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_trim, y_trim, test_size=0.2, random_state=42
)

## Fitting the Model

In [24]:
# Fitting the model 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

xgb = XGBClassifier(n_estimators=100, max_depth=5)
xgb.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Checking Model Performance

In [29]:
# Classifying
y_pred_test = xgb.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)
print("acc_test: ", acc_test)

# validation check
y_pred_val = xgb.predict(X_val)
acc_val = accuracy_score(y_val, y_pred_val)
print("acc_val: ", acc_val)

acc_test:  0.9974426162429278
acc_val:  0.9973364717921224


In [31]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(y_test, y_pred_test))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_test))

print(classification_report(y_val, y_pred_val))
print("ROC-AUC:", roc_auc_score(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    127291
           1       1.00      1.00      1.00    126875

    accuracy                           1.00    254166
   macro avg       1.00      1.00      1.00    254166
weighted avg       1.00      1.00      1.00    254166

ROC-AUC: 0.9974452109945664
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   5718458
           1       1.00      1.00      1.00   5719003

    accuracy                           1.00  11437461
   macro avg       1.00      1.00      1.00  11437461
weighted avg       1.00      1.00      1.00  11437461

ROC-AUC: 0.9973363978865064


# Genetic Programming 
## Estimating the Symbolic Mathematical Expression for the XGBoost Model 

In [32]:
!pip install gplearn

In [33]:
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.metrics import mean_squared_error

# Train GP to mimic XGB predictions (regression task)
gp = SymbolicRegressor(
    population_size=1000,
    generations=20,
    stopping_criteria=0.01,
    p_crossover=0.7,
    function_set=['add', 'sub', 'mul', 'div', 'sin', 'cos', 'sqrt', 'log', 'abs'],
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.9,
    verbose=1,
    parsimony_coefficient=0.001,
    random_state=42,
    n_jobs=-1
)

In [34]:
xgb_preds = xgb.predict(X_train)
gp.fit(X_train, xgb_preds)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


   0     8.52      9.19687e+10       24         0.197913         0.197704     14.09m
   1     6.40      2.57473e+10       13         0.196945         0.197841     10.84m
   2     6.79      2.45873e+10       11         0.113337         0.112517     10.78m
   3    14.11      1.35199e+06       15         0.100787         0.100968     15.35m
   4     8.84          30641.2       18        0.0920474        0.0912784      9.20m
   5    11.35          38793.2       20        0.0918058        0.0926026     11.66m
   6    15.54      1.29934e+08       12       0.00899348       0.00934423     13.55m


SymbolicRegressor(function_set=['add', 'sub', 'mul', 'div', 'sin', 'cos',
                                'sqrt', 'log', 'abs'],
                  max_samples=0.9, n_jobs=-1, p_crossover=0.7,
                  p_hoist_mutation=0.05, p_point_mutation=0.1,
                  p_subtree_mutation=0.1, random_state=42,
                  stopping_criteria=0.01, verbose=1)

In [36]:
# Printing the Estimated Mathematical Expresion
print("Evolved Function:", gp._program)

Evolved Function: div(log(X7), div(sub(X2, X1), abs(sin(div(X2, X1)))))


## GP Model Performance

In [35]:
#Chekcing the result
y_pred_gp = gp.predict(X_test)
r2_pred_gp = r2_score(y_test, y_pred_gp)
y_val_gp = gp.predict(X_val)
r2_val_gp = r2_score(y_val, y_val_gp)
print(f"\nTest R²: {r2_pred_gp}, {r2_val_gp}")


Test R²: 0.9712785362487173, 0.9701125974745478
